## 1. Załadowanie bibliotek

In [101]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.preprocessing import StandardScaler, FunctionTransformer
import category_encoders as ce
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from datetime import datetime
from sklearn.metrics import mean_squared_error

## 2. Zbiór danych "apartments"

In [56]:
dtrain = pd.read_csv("apartments.csv")
dtest = pd.read_csv("apartments_test.csv")

target = "m2.price"
predictors = [i for i in dtrain.columns if i not in [target]]

### 2.1. Sprawdzenie konieczności imputacji

In [57]:
## Brak braków danych mozemy przejść do encodingu zmiennych kategorycznych
dtrain.isnull().values.any()

False

### 2.2. Podział danych na zbiór testowy oraz treningowy

In [65]:
# Sprawdzmy jak duzo jest uniklanych wartości w kolumnei district
dtrain["district"].unique()
# Jest ich na tyle nieduzo, ze mozęmy skorzystać z OneHotEncodingu

X_train = dtrain[predictors]
Y_train = dtrain[target]
X_test = dtest[predictors]
Y_test = dtest[target]

# Kolumny z danymi numerycznymi
num = X_train.select_dtypes(include=['float64', 'int']).columns

# Kolumny z danymi kategorycznymi
cat = X_train.select_dtypes(include=['object']).columns.tolist()

### 2.3. SVR ze StandardScaler()

In [66]:
# Funkcja służąca do zbudowania pipeline'a, który w kolejnych krokach będzie miał encoding, scalling oraz 
# klasyfikator - dzięki temu pozbywamy sie dataleak'u przy kroswalidacji, ponieważ dla każdego podziału 
# kroswalidacji będzie wykonywany scalling tylko na zbiorze treningowym, nie biorąc pod uwagę testowego
def makePipeline(cat_indices, num_indices, classificator, encoder, scaled = False):
    
    if scaled == True:
        pipeline = Pipeline(steps = [
        ('feature_processing', FeatureUnion(transformer_list = [
                ('categorical', FunctionTransformer(lambda data: data[cat_indices])),

                #numeric
                ('numeric', Pipeline(steps = [
                    ('select', FunctionTransformer(lambda data: data[num_indices])),
                    ('scale', StandardScaler())
                            ]))
            ])),
        ('encoder', encoder),
        ('classifier', classificator)
        ]
    )
    else:
        pipeline = Pipeline([
        ('encoder', encoder),
        ('classifier', classificator)
        ])
    return pipeline

# Funkcja, która zmierzy nam ilość czasu potrzebną na wykonanie danej funkcji

def timer(start_time = None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [114]:
# Stworzenei Pipeline'a z domyślnym klasyfiaktorem
svr_classifier = svm.SVR()
svr_scaled_pipeline = makePipeline(cat, num, svr_classifier, ce.OneHotEncoder(), True)

params = {
    "classifier__C": [i/100 for i in range(1,100000,100)],
    "classifier__gamma": ["scale","auto"],
    "classifier__degree": [i for i in range(1,10,1)]
}

# Jako metryke wzgledem ktorej bedziemy powownywac przyjmiemy RMSE
svr_scaled_random_search = RandomizedSearchCV(svr_scaled_pipeline, param_distributions = params, n_iter = 1000,
                                      scoring = 'neg_root_mean_squared_error', n_jobs = -1, cv = 10,
                                      random_state = 1001)

start_time = timer(None) # aktualna godzina
svr_scaled_random_search.fit(X_train, Y_train)
timer(start_time) # ile czasu minęło od start_time
print(svr_scaled_random_search.best_score_)
print(svr_scaled_random_search.best_params_)


 Time taken: 0 hours 6 minutes and 17.46 seconds.
-45.98750420537989
{'classifier__gamma': 'scale', 'classifier__degree': 9, 'classifier__C': 21.01}


### 2.4. SVR bez StandardScaler()

In [82]:
# Stworzenei Pipeline'a z domyślnym klasyfiaktorem
svr_classifier = svm.SVR()
svr_pipeline = makePipeline(cat, num, svr_classifier, ce.OneHotEncoder(), False)

# Jako metryke wzgledem ktorej bedziemy powownywac przyjmiemy RMSE
svr_random_search = RandomizedSearchCV(svr_pipeline, param_distributions = params, n_iter = 1000,
                                      scoring = 'neg_root_mean_squared_error', n_jobs = -1, cv = 10,
                                      random_state = 1001)

start_time = timer(None) # aktualna godzina
svr_random_search.fit(X_train, Y_train)
timer(start_time) # ile czasu minęło od start_time
print(svr_random_search.best_score_)
print(svr_random_search.best_params_)


 Time taken: 0 hours 5 minutes and 7.42 seconds.
-772.0637357973118
{'classifier__gamma': 'auto', 'classifier__degree': 7, 'classifier__C': 999.01}


### 2.5. Porównanie wyników

In [95]:
default_model = makePipeline(cat, num, svm.SVR(), ce.OneHotEncoder(), False)
tuned_model = svr_random_search.best_estimator_
scaled_model = svr_scaled_random_search.best_estimator_

default_model.fit(X_train, Y_train)
tuned_model.fit(X_train, Y_train)
scaled_model.fit(X_train, Y_train)

y_pred_default = default_model.predict(X_test)
y_pred_tuned = tuned_model.predict(X_test)
y_pred_scaled = scaled_model.predict(X_test)

mean_squared_error
pd.DataFrame({"Model" : ["Bez strojenia", "Strojenie", "Strojenie + StandradScaler"],
             "RMSE": [mean_squared_error(Y_test, y_pred_default, squared = False),
                      mean_squared_error(Y_test, y_pred_tuned, squared = False),
                      mean_squared_error(Y_test, y_pred_scaled, squared = False)]
            })

,Model,RMSE
0,Bez strojenia,909.021810
1,Strojenie,793.275074
2,Strojenie + StandradScaler,158.839379


## 3. Zbiór danych "forestfires"

In [105]:
df = pd.read_csv("forestfires.csv")
target = "area"
predictors = [i for i in df.columns if i not in [target]]

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       517 non-null    int64  
 1   Y       517 non-null    int64  
 2   month   517 non-null    object 
 3   day     517 non-null    object 
 4   FFMC    517 non-null    float64
 5   DMC     517 non-null    float64
 6   DC      517 non-null    float64
 7   ISI     517 non-null    float64
 8   temp    517 non-null    float64
 9   RH      517 non-null    int64  
 10  wind    517 non-null    float64
 11  rain    517 non-null    float64
 12  area    517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


### 3.1. Sprawdzenie konieczności imputacji

In [103]:
## Brak braków danych mozemy przejść do encodingu zmiennych kategorycznych
df.isnull().values.any()

False

### 3.2. Podział danych na zbiór testowy oraz treningowy

In [106]:
# Sprawdzmy jak duzo jest uniklanych wartości w kolumnei district
dtrain["district"].unique()
# Jest ich na tyle nieduzo, ze mozęmy skorzystać z OneHotEncodingu

X_train, X_test, Y_train, Y_test = train_test_split(df.drop(target, axis=1), df[target])

# Kolumny z danymi numerycznymi
num = X_train.select_dtypes(include=['float64', 'int']).columns

# Kolumny z danymi kategorycznymi
cat = X_train.select_dtypes(include=['object']).columns.tolist()

### 3.3. SVR ze StandardScaler()

In [108]:
# Stworzenei Pipeline'a z domyślnym klasyfiaktorem
svr_classifier = svm.SVR()
svr_scaled_pipeline = makePipeline(cat, num, svr_classifier, ce.OneHotEncoder(), True)

# Jako metryke wzgledem ktorej bedziemy powownywac przyjmiemy RMSE
svr_scaled_random_search = RandomizedSearchCV(svr_scaled_pipeline, param_distributions = params, n_iter = 1000,
                                      scoring = 'neg_root_mean_squared_error', n_jobs = -1, cv = 10,
                                      random_state = 1001)

start_time = timer(None) # aktualna godzina
svr_scaled_random_search.fit(X_train, Y_train)
timer(start_time) # ile czasu minęło od start_time
print(svr_scaled_random_search.best_score_)
print(svr_scaled_random_search.best_params_)


 Time taken: 0 hours 6 minutes and 16.67 seconds.
-45.98750420537989
{'classifier__gamma': 'scale', 'classifier__degree': 9, 'classifier__C': 21.01}


### 3.4. SVR bez StandradScaler()

In [109]:
# Stworzenei Pipeline'a z domyślnym klasyfiaktorem
svr_classifier = svm.SVR()
svr_pipeline = makePipeline(cat, num, svr_classifier, ce.OneHotEncoder(), False)

# Jako metryke wzgledem ktorej bedziemy powownywac przyjmiemy RMSE
svr_random_search = RandomizedSearchCV(svr_pipeline, param_distributions = params, n_iter = 1000,
                                      scoring = 'neg_root_mean_squared_error', n_jobs = -1, cv = 10,
                                      random_state = 1001)

start_time = timer(None) # aktualna godzina
svr_random_search.fit(X_train, Y_train)
timer(start_time) # ile czasu minęło od start_time
print(svr_random_search.best_score_)
print(svr_random_search.best_params_)


 Time taken: 0 hours 3 minutes and 44.55 seconds.
-45.56375710761044
{'classifier__gamma': 'auto', 'classifier__degree': 8, 'classifier__C': 10.01}


### 3.5. Porównanie wyników

In [111]:
default_model = makePipeline(cat, num, svm.SVR(), ce.OneHotEncoder(), False)
tuned_model = svr_random_search.best_estimator_
scaled_model = svr_scaled_random_search.best_estimator_

default_model.fit(X_train, Y_train)
tuned_model.fit(X_train, Y_train)
scaled_model.fit(X_train, Y_train)

y_pred_default = default_model.predict(X_test)
y_pred_tuned = tuned_model.predict(X_test)
y_pred_scaled = scaled_model.predict(X_test)

mean_squared_error
pd.DataFrame({"Model" : ["Bez strojenia", "Strojenie", "Strojenie + StandradScaler"],
             "RMSE": [mean_squared_error(Y_test, y_pred_default, squared = False),
                      mean_squared_error(Y_test, y_pred_tuned, squared = False),
                      mean_squared_error(Y_test, y_pred_scaled, squared = False)]
            })

,Model,RMSE
0,Bez strojenia,32.625247
1,Strojenie,32.090364
2,Strojenie + StandradScaler,32.265921


## 4. Podsumowanie wyników

### 4.1. Zbiór "apartments"

To co momentalnie rzuca się w oczy to fakt, że skalowanie danych za pomocą funkcji StandradScaler() zdecydowanie poprawiłu rezultaty - znacząco zmniejszyło RMSE. Jeżeli chodzi o samą wartość RMSE można powiedzieć, że jest ona satysfakcjonująca. Biorąc pod uwagę rząd wielkości zmiennej celu, której średnia wynosi 3587, a odchylenei standradowe 906, to wynik po skalowaniu danych jest akceptowalny. Kolejna rzecz, którą można zauważyc to fakt, że podczas RandomizedSearchCV() dla danych nieskalowanych została wzięta największa możliwa wartość parametru c, co oznacza, że prawdopodobnie, gdyby zwiększyć jeszcze zakres to większa wartość C mogłaby dać jeszcze lepsze rezultaty.

### 4.2. Zbiór "forestfire"

Jeżeli chodzi o zbiór forestfire to okazał się on bardzo nietrafiony dla tego modelu. Sam zbió© danych jest dość ubogi - zawiera tylko 517 wierszy. Ponadto zmienan celu jest "niezbalanasowana", bo mimo, że jest zmienną ciagłą to i tak ponad ćwierć jej obserwacji wynosi 0 - co za tym idzie jest bardzo spore odchylenei standardowe. Myślę, ze te m.in. te 2 rzeczy są odpowiedzialne za marne wyniki tego algorytmu. Co ciekawe nie widać róznic w algorytmie dla danych przeskalowanych, a tych nieprzesaklowanych. Wynik jest mało satysfakcjonujący, biorąc pod uwagę, że średnia zmiennej celu to zaledwie 12.84. ZArówno jest to przykład bardzo trudnego zbioru do analizy, a zarazem takiego, na którym metoda SVR nie jest zbyt wydajna.